In [1]:
from tqdm.notebook import tqdm
import codecs
import pandas as pd
tqdm.pandas()
import string
from multiprocessing import  Pool
import numpy as np
import random

# parallelising Dataframe

In [2]:
def parallelize_dataframe(df, func, n_cores=2):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [3]:
# from google.colab import drive
import os
# drive.mount('/content/drive')
# os.chdir("./drive/MyDrive/NLP Project")
os.chdir("./")

In [4]:
os.listdir()

['__pycache__',
 'NLP Full.ipynb',
 'dummy',
 'Post_processing and Classiffication.ipynb',
 'embedder.py',
 '.ipynb_checkpoints',
 'splitData']

# data Download

In [5]:
# !unzip  "/content/drive/MyDrive/NLP Project/1.zip" -d "/content/drive/MyDrive/NLP Project/unzipData/"
# datafolder=os.listdir("./unzipData/")
# import os, zipfile

# # dir_name = 'C:\\SomeDirectory'
# extension = ".zip"

# # os.chdir(dir_name) # c?hange directory from working dir to dir with files?

# for item in datafolder: # loop through items in dir
#     if item.endswith(extension): # check for ".zip" extension
#         zip_ref = zipfile.ZipFile("./unzipData/"+item) # create zipfile object
#         zip_ref.extractall("./FinalDataset") # extract file to dir
#         zip_ref.close() # close file

# install required packages

In [6]:
!pip install demoji

#loading embedders

In [7]:
from embedder import GoogleUniversalSentenceEncoder

2022-04-26 13:47:04.985443: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-26 13:47:04.985501: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-04-26 13:47:07.970780: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 13:47:07.971064: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-26 13:47:07.971137: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.s

In [14]:
# guse = GoogleUniversalSentenceEncoder()
import tensorflow_hub as hub
import tensorflow_text
    
# The 16-language multilingual module is the default but feel free
# to pick others from the list and compare the results.
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual/3' #@param ['https://tfhub.dev/google/universal-sentence-encoder-multilingual/3', 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3']

model = hub.load(module_url)

def embed_text(input):
    return model(input)[-1]

def embed_text_list(input):
    return model(input).numpy()

# preprocessing functions

In [15]:
import string
import re
import demoji
def preprocessText(text):
    text = str(text)
    # remove url from text
    text = re.sub(r'http\S+', '', text)
    # remove the punctuations and emojis from the text
    # text = emoji.replace_emoji(text, '')
    text = demoji.replace(text,"")
    # text = "".join([c for c in text if c not in string.punctuation])
    text=str(text).translate(str.maketrans('','',string.punctuation))
    return text

def preprocessDF(df):
    df=df.loc[:,["mid","text","created_at","deleted_last_seen","permission_denied"]].fillna(False)
    return df

# encode and save to solve issue of the utf8 read error

In [16]:
# for csv in tqdm(os.listdir("./FinalDataset/")):
#   with codecs.open("./FinalDataset/"+csv, 'r', encoding='utf-8',errors='ignore') as fdata:
#     csvData=fdata.read()
#   with open("./encodedDatasetV2/"+"p_"+csv, 'w', encoding='utf8') as f:
#     f.write(csvData)

# del csvData
# del f

# find all the censored data

In [17]:
def getDataSplit(df):
    df_censored=df.loc[df["permission_denied"]==True]
    df_not_censored=df.loc[df["permission_denied"]==False]
    return df_censored,df_not_censored

In [18]:
def parallelPreprocess(df):
    df["text"] = df["text"].progress_apply(lambda x:preprocessText(str(x)))
    return df

In [19]:
data_folder="/home/edwinjose/Documents/NLP Project/dataset/encodedDatasetV2/"
# os.listdir(data_folder)

In [20]:
os.listdir()

['__pycache__',
 'NLP Full.ipynb',
 'dummy',
 'Post_processing and Classiffication.ipynb',
 'embedder.py',
 '.ipynb_checkpoints',
 'splitData']

In [27]:

for csv in tqdm(os.listdir(data_folder)):
    df=pd.read_csv(data_folder+csv)
    df_clean=preprocessDF(df)
    df_censored,_ = getDataSplit(df_clean)
    df_censored = parallelize_dataframe(df = df_censored, func = parallelPreprocess, n_cores=7 )
    df_censored.to_csv('./splitData/censored.csv', mode='a', index=False)
    # df_not_censored.to_csv('./splitData/not_censored.csv', mode='a', index=False, )
    del df_censored
    del df_clean
  


  0%|          | 0/52 [00:00<?, ?it/s]

/tmp/ipykernel_9230/533125187.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(data_folder+csv)
/tmp/ipykernel_9230/533125187.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(data_folder+csv)
/tmp/ipykernel_9230/533125187.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(data_folder+csv)
/tmp/ipykernel_9230/533125187.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(data_folder+csv)
/tmp/ipykernel_9230/533125187.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(data_folder+csv)
/tmp/ipykernel_9230/533125187.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.